In [ ]:
%pip install 'aif360[Reductions]'

In [ ]:
%pip install tensorflow

In [52]:
from aif360.algorithms.inprocessing import PrejudiceRemover,AdversarialDebiasing
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import joblib
import re
import sys
sys.path.insert(0, '../')
import numpy as np

# Datasets
from aif360.datasets import MEPSDataset19
from aif360.datasets import MEPSDataset20
from aif360.datasets import MEPSDataset21
from aif360.datasets import GermanDataset


# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler


from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

#inprocessing technique 

from aif360.algorithms.inprocessing import MetaFairClassifier
from aif360.algorithms.inprocessing import GerryFairClassifier

In [5]:
dataset_orig_panel19 = MEPSDataset19()

In [6]:
dataset_orig_panel19_train, dataset_orig_panel19_val = dataset_orig_panel19.split([0.7], shuffle=True)

In [7]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                    dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]]

In [8]:
def test(dataset, model, thresh, unprivileged_groups, privileged_groups):
    y_val_pred_prob = model.predict(dataset).scores
    pos_ind = 0  # Assuming scores are for the favorable outcome

    # Ensure y_val_pred_prob is two-dimensional
    if y_val_pred_prob.ndim == 1:
        # If y_val_pred_prob is one-dimensional, use it directly
        y_val_pred = (y_val_pred_prob > thresh).astype(int)
    else:
        # Use pos_ind to index the second dimension
        y_val_pred = (y_val_pred_prob[:, pos_ind] > thresh).astype(int)

    # Continue with the evaluation using y_val_pred
    # Calculate accuracy, fairness metrics, etc., based on y_val_pred

    return {
        'accuracy': None,  # Replace with actual accuracy calculation
        'fairness_metric': None  # Replace with actual fairness metric calculation
    }


In [10]:
features = dataset_orig_panel19_train.features
label = dataset_orig_panel19_train.labels
feature_names = dataset_orig_panel19_train.feature_names
dataset_orig_panel19_train_df = pd.DataFrame(features, columns=feature_names)
dataset_orig_panel19_train_df['RACE2']=dataset_orig_panel19_train_df['RACE']
dataset_orig_panel19_train_df.drop('RACE',axis=1,inplace=True)
dataset_orig_panel19_train_df['label'] = label

In [11]:
dataset_orig_panel19_train_df

,AGE,PCS42,MCS42,K6SUM42,REGION=1,REGION=2,REGION=3,REGION=4,SEX=1,SEX=2,...,POVCAT=1,POVCAT=2,POVCAT=3,POVCAT=4,POVCAT=5,INSCOV=1,INSCOV=2,INSCOV=3,RACE2,label
0,28.0,56.71,62.39,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,20.0,52.40,62.66,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,72.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,38.0,-1.00,-1.00,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,79.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11076,25.0,48.86,58.43,4.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11077,9.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11078,16.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11079,5.0,-1.00,-1.00,-1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [12]:
from aif360.datasets import StandardDataset

# Define which column is the label and which one is the protected attribute
label_name = 'label'  # or whatever the name of your label column is
protected_attribute_name = 'RACE2'  # or your protected attribute column name

# Create the aif360 dataset
aif360_dataset = StandardDataset(
    df=dataset_orig_panel19_train_df,
    label_name=label_name,
    favorable_classes=[1],  # assuming 1 is the favorable class
    protected_attribute_names=[protected_attribute_name],
    privileged_classes=[[1.0]],  # assuming 'White' is the privileged class
)


In [13]:
aif360_dataset_train,aif360_dataset_test=aif360_dataset.split([0.7], shuffle=True)

In [14]:
aif360_dataset_train

               instance weights features                                 \
                                                                          
                                     AGE  PCS42  MCS42 K6SUM42 REGION=1   
instance names                                                            
4466                        1.0      1.0  -1.00  -1.00    -1.0      0.0   
8666                        1.0     35.0  58.84  51.24     1.0      0.0   
2903                        1.0     10.0  -1.00  -1.00    -1.0      0.0   
1551                        1.0     23.0  56.15  57.16     0.0      0.0   
10019                       1.0     58.0  52.94  37.17     7.0      0.0   
...                         ...      ...    ...    ...     ...      ...   
923                         1.0     19.0  46.70  57.24     0.0      0.0   
1976                        1.0     11.0  -1.00  -1.00    -1.0      0.0   
4842                        1.0     19.0  56.71  62.39     1.0      1.0   
7799                     

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

pr=PrejudiceRemover(sensitive_attr='RACE2', eta=25.0)
dset_scaled_trn = aif360_dataset_train.copy()
dset_scaled_trn.features = scaler.fit_transform(dset_scaled_trn.features)

pr_fitted = pr.fit(dset_scaled_trn)
print(pr_fitted)
accs = []
thresholds = np.linspace(0.01, 0.5, 10)

dset_val = aif360_dataset_test.copy()
dset_val.features = scaler.transform(dset_val.features)

##################### STEP 1 TRAINING WITH INPROCESSING #####################
pr_pred_prob = pr_fitted.predict(dset_val).scores
print(pr_pred_prob)
##################### STEP 2 PICKING THRESHOLD WITH VALIDATION DATA #####################
for threshold in thresholds:
    dset_val_pred = dset_val.copy()
    dset_val_pred.labels = (pr_pred_prob[:, 0] > threshold).astype(np.float64)

    metric = ClassificationMetric(
                dset_val, dset_val_pred,
                unprivileged_groups = unprivileged_groups,
                privileged_groups=privileged_groups)
    accs.append((metric.true_positive_rate() + metric.true_negative_rate()) / 2)


pr_val_best_idx = np.argmax(accs)
best_threshold = thresholds[pr_val_best_idx]

##################### STEP 3 TEST DATA #####################
dset_tst = dataset_orig_panel19_val.copy()
dset_tst.features = scaler.transform(dset_tst.features)

pr_pred_prob = pr_fitted.predict(dset_tst).scores


dset_tst_pred = dset_tst.copy()
dset_tst_pred.labels = (pr_pred_prob[:, 0] > best_threshold).astype(np.float64)

metric = ClassificationMetric(
            dset_tst, dset_tst_pred,
            unprivileged_groups = unprivileged_groups,
            privileged_groups   = privileged_groups)
test_acc = (metric.true_positive_rate() + metric.true_negative_rate()) / 2 ## no built in balanced error rate
test_disp_impact = metric.disparate_impact()

print("Testing accuracy with ETA %0.2f = %0.2f\n Disparate impact %0.2f" % (25.0, test_acc, test_disp_impact))

In [ ]:
pr_orig_best_ind = np.argmax(val_metrics['bal_acc'])

In [ ]:
print(val_metrics['bal_acc'])
print(val_metrics['bal_acc'].shape)


In [16]:
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

# Now you can use tf.Session like in TensorFlow 1.x
session = tf.compat.v1.Session()

debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          num_epochs=10,
                          debias=True,
                          sess=session)

debiased_model.fit(dataset_orig_panel19_train)

#fair = get_fair_metrics_and_plot(data_orig_test, debiased_model, plot=False, model_aif=True)
data_pred = debiased_model.predict(dataset_orig_panel19_val)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
2024-04-07 19:38:47.458787: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


epoch 0; iter: 0; batch classifier loss: 1.064564; batch adversarial loss: 0.844722
epoch 1; iter: 0; batch classifier loss: 0.492836; batch adversarial loss: 0.762379
epoch 2; iter: 0; batch classifier loss: 0.422149; batch adversarial loss: 0.702992
epoch 3; iter: 0; batch classifier loss: 0.309038; batch adversarial loss: 0.677298
epoch 4; iter: 0; batch classifier loss: 0.360344; batch adversarial loss: 0.676065
epoch 5; iter: 0; batch classifier loss: 0.325244; batch adversarial loss: 0.623558
epoch 6; iter: 0; batch classifier loss: 0.259387; batch adversarial loss: 0.650880
epoch 7; iter: 0; batch classifier loss: 0.348109; batch adversarial loss: 0.688949
epoch 8; iter: 0; batch classifier loss: 0.364343; batch adversarial loss: 0.641030
epoch 9; iter: 0; batch classifier loss: 0.395801; batch adversarial loss: 0.633827


In [17]:
metric_transf_panel19_train = BinaryLabelDatasetMetric(
data_pred,
unprivileged_groups=unprivileged_groups,
privileged_groups=privileged_groups)
explainer_transf_panel19_train = MetricTextExplainer(metric_transf_panel19_train)
test_results_ad=[explainer_transf_panel19_train.mean_difference()
               ,explainer_transf_panel19_train.consistency()
               ,explainer_transf_panel19_train.statistical_parity_difference()
               ,explainer_transf_panel19_train.disparate_impact()]

In [18]:
test_results_ad

['Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.09253696095777574',
 'Consistency (Zemel, et al. 2013): [0.93821857]',
 'Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.09253696095777574',
 'Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.49708199611485']

In [79]:
MC = MetaFairClassifier(tau=0, sensitive_attr="RACE", type="fdr")

In [80]:
mc_model = MC.fit(dataset_orig_panel19_train)

In [81]:
dataset_transf_panel19_train_MC = mc_model.predict(dataset_orig_panel19_train)

In [77]:
dataset_transf_panel19_test_MC = mc_model.predict(dataset_orig_panel19_val)

In [60]:
classified_metric_bias_test = BinaryLabelDatasetMetric(dataset_transf_panel19_test_MC,
                                                   unprivileged_groups=unprivileged_groups,
                                                   privileged_groups=privileged_groups)
explainer_transf_panel19_train = MetricTextExplainer(metric_transf_panel19_train)
test_results_mc=[explainer_transf_panel19_train.mean_difference()
               ,explainer_transf_panel19_train.consistency()
               ,explainer_transf_panel19_train.statistical_parity_difference()
               ,explainer_transf_panel19_train.disparate_impact()]

In [83]:
test_results_mc

['Mean difference (mean label value on unprivileged instances - mean label value on privileged instances): -0.09253696095777574',
 'Consistency (Zemel, et al. 2013): [0.93821857]',
 'Statistical parity difference (probability of favorable outcome for unprivileged instances - probability of favorable outcome for privileged instances): -0.09253696095777574',
 'Disparate impact (probability of favorable outcome for unprivileged instances / probability of favorable outcome for privileged instances): 0.49708199611485']

In [84]:
GC = GerryFairClassifier(C= 100, printflag=True, gamma=.005, fairness_def='FP',
             max_iters=10, heatmapflag=False)

In [86]:
# fit method
gfc_model = GC.fit(dataset_orig_panel19_train, early_termination=True)

# predict method. If threshold in (0, 1) produces binary predictions
dataset_yhat = gfc_model.predict(dataset_orig_panel19_train, threshold=False)

iteration: 1, error: 0.13437415395722407, fairness violation: 0.005949616964211739, violated group size: 0.2653190145293746
iteration: 2, error: 0.15251331107300786, fairness violation: 0.0029764690924395944, violated group size: 0.2653190145293746
iteration: 3, error: 0.15846945221550404, fairness violation: 0.002005843173648617, violated group size: 0.2653190145293746
iteration: 4, error: 0.16131215594260445, fairness violation: 0.0015132866024116579, violated group size: 0.2653190145293746
iteration: 5, error: 0.16296363144120565, fairness violation: 0.0012300066827493246, violated group size: 0.2653190145293746
iteration: 6, error: 0.16401949282555725, fairness violation: 0.001041153402974436, violated group size: 0.2653190145293746
iteration: 7, error: 0.1646576508051104, fairness violation: 0.0009021189963686753, violated group size: 0.5610504467105857
iteration: 8, error: 0.16509114700839275, fairness violation: 0.0008131607202641582, violated group size: 0.5610504467105857
iter

In [87]:
# auditing 
from aif360.algorithms.inprocessing.gerryfair.clean import array_to_tuple

gerry_metric = BinaryLabelDatasetMetric(dataset_orig_panel19_train)
gamma_disparity = gerry_metric.rich_subgroup(array_to_tuple(dataset_yhat.labels), 'FP')
print(gamma_disparity)

0.0007507787405605569


In [92]:
from sklearn import svm
from sklearn import tree
from sklearn.kernel_ridge import KernelRidge
from sklearn import linear_model
import pickle

In [93]:
# set to 10 iterations for fast running of notebook - set >= 1000 when running real experiments
# tests learning with different hypothesis classes
pareto_iters = 10
def multiple_classifiers_pareto(dataset, gamma_list=[0.002, 0.005, 0.01], save_results=True, iters=pareto_iters):

    ln_predictor = linear_model.LinearRegression()
    svm_predictor = svm.LinearSVR()
    tree_predictor = tree.DecisionTreeRegressor(max_depth=3)
    kernel_predictor = KernelRidge(alpha=1.0, gamma=1.0, kernel='rbf')
    predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': iters},
                       'SVR': {'predictor': svm_predictor, 'iters': iters},
                       'Tree': {'predictor': tree_predictor, 'iters': iters},
                       'Kernel': {'predictor': kernel_predictor, 'iters': iters}}
    #predictor_dict = {'Linear': {'predictor': ln_predictor, 'iters': iters}}

    results_dict = {}

    for pred in predictor_dict:
        print('Curr Predictor: {}'.format(pred))
        predictor = predictor_dict[pred]['predictor']
        max_iters = predictor_dict[pred]['iters']
        fair_clf = GerryFairClassifier(C=100, printflag=True, gamma=1, predictor=predictor, max_iters=max_iters)
        fair_clf.printflag = False
        fair_clf.max_iters=max_iters
        errors, fp_violations, fn_violations = fair_clf.pareto(dataset, gamma_list)
        results_dict[pred] = {'errors': errors, 'fp_violations': fp_violations, 'fn_violations': fn_violations}
    if save_results:
        print('path:'+'results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb')
        pickle.dump(results_dict, open('results_dict_' + str(gamma_list) + '_gammas' + str(gamma_list) + '.pkl', 'wb'))

multiple_classifiers_pareto(dataset_orig_panel19_train)

Curr Predictor: Linear
path:results_dict_[0.002, 0.005, 0.01]_gammas[0.002, 0.005, 0.01].pkl wb
